# Putting It Together


* The following example takes all of the different things we have learned and combines them together into a program
* This program will processes [QuadraSTAR](https://lucidea.com/cuadrastar/) data files and converts the *narrow* data format into a *wide* data format (see [wikipedia for more discussion](https://en.wikipedia.org/wiki/Wide_and_narrow_data). 
* QuadraSTAR records looks like this:

```
--REC--4
IU     10/09/2003
NAME   Egger, J.C. Jr
REFNO  0890-0-01
SERV   USAF
TYPE   Message
DOR    94 08 19
DOI    67 11 03
IIRN   191300zAUG94
ORG    SECDEF
SUBJ   Public Affairs-POW/MIA-Press Guidance for identification of remains-Cambodia and Vietnam-August 94.
CAT    Casualty files
PAGE   11-15
REEL   512
REELC  1
FOLD   115-8
CNTRY  Cambodia
CNTRY  North Vietnam, pre-1975
KYWDS  Remains
KYWDS  Aircraft downed
KYWDS  Quang Tri Province
RSCHR  jego
COMM   The remains of three American servicemen previously unaccounted for in Indochina, have been identified.
SUBJ   Public Affairs-POW/MIA-Press Guidance for identification of remains-Cambodia and Vietnam-August 94.
CAT    Casualty files
PAGE   11-15
REEL   512
REELC  1
FOLD   115-8
CNTRY  Cambodia
CNTRY  North Vietnam, pre-1975
KYWDS  Remains
KYWDS  Aircraft downed
KYWDS  Quang Tri Province
RSCHR  jego
COMM   The remains of three American servicemen previously unaccounted for in Indochina, have been identified.
```

* Each record is a series of attribute value pairs split over a series of lines separated by `--REC-<num>`
* The QuadraSTAR data is very inconvenient because it doesn't include the record ID in each line
* This means we have to keep track of records as we parse through the file
* We want it to end up as a CSV file with one record per row and all the attributes as columns

```
PWMIA_4,PWMIA,4,,Casualty files,"Cambodia; North Vietnam, pre-1975; Cambodia; North Vietnam, pre-1975","The remains of three American servicemen previously unaccounted for in Indochina, have been identified.",,67 11 03,94 08 19,,115-8,191300zAUG94,,10/09/2003,Remains; Aircraft downed; Quang Tri Province; Remains; Aircraft downed; Quang Tri Province,"Egger, J.C. Jr",,SECDEF,11-15,,,,512,1,0890-0-01,,jego,,USAF,,Public Affairs-POW/MIA-Press Guidance for identification of remains-Cambodia and Vietnam-August 94.,,Message,
```
* We will have to make two passes through the data because we need to know all the headers in advance
* The hard part, beyond knowing the Python commands, is thinking about how to assemble the computational logic to perform the processing
    * This *computational thinking* takes time and practice to learn
    * There are also many ways to compose this program, smarter programmers might have more clever ways of performing this data processing


## Workflow
* Get a list of a QuadraSTAR files in the data directory
* Parse the files and generate a list of headers
    * Save that list to a file
* Create a CSV file for saving the transformed records
* Loop through every line of every data files
    * Parse the line into header and value
    * If the line is a REC start building a dictionary for the row
    * When the record is complete write a row to the CSV file


## Loading Libraries

* We are only going to use two Python libraries in this program
* `csv` to help writing the CSV file (much better than doing it manually)
* `pathlib` to help locating the source data files

In [1]:
# Load the libraries
import csv
from pathlib import Path

## Setting Variables

* To make this program easy to change we will put some filepaths up top
* This makes the script a bit more flexible if the files are in different directories

In [2]:
data_folder = "clean_star/"
header_file_name = "headers.txt"
data_file_name = "data.csv"

# Generate a list of data files using the pathlibe function
# Explicitly make it a list rather than a generator
file_list = list(Path(data_folder).glob("*.txt"))
print(file_list)

[PosixPath('clean_star/PWMIA.txt'), PosixPath('clean_star/PWMIA03.txt'), PosixPath('clean_star/PWMIA04.txt'), PosixPath('clean_star/PWMIA05.txt')]


## Reading Headers

* Before we can start writing the CSV file we need to generate a list of the headers
* This involves quickly reading through the files and creating a list of 
* 

In [ ]:
# Create a little function to use in a list comprehension
# for processing lines
def process_line(line):
    # Quadraster is a fixed width format, the first 6 characters are the header
    # We can slice the header from the line
    # Also replace dashes with nothing and strip extra whitespace
    header = line[0:6].replace("-", "").strip()
    return header

# Loop over the list of files
for file in file_list:
    # Open the file in read-only mode
    with open(file, "r") as f:
        # Use a set comprehension to make a list of unique header values
        fields = {process_line(line) for line in f}

# Display the identified fields
fields

* This code has a BUG!
* It is a *semantic* bug in the logic of the program
    * Meaning it runs fine but doesn't produce expected behavior
* Can you figure out the bug?

In [5]:
all_fields = set()

# Create a little function to use in a list comprehension
# for processing lines
def process_line(line):
    # Quadraster is a fixed width format, the first 6 characters are the header
    # We can slice the header from the line
    # Also replace dashes with nothing and strip extra whitespace
    header = line[0:6].replace("-", "").strip()
    return header

# Loop over the list of files
for file in file_list:
    # Open the file in read-only mode
    with open(file, "r") as f:
        # Use a set comprehension to make a list of unique header values
        fields = {process_line(line) for line in f}
        all_fields.update(fields)

# Display the identified fields
all_fields

{'CAPT',
 'CAT',
 'CNTRY',
 'COMM',
 'COND',
 'DOI',
 'DOR',
 'EDIT',
 'FOLD',
 'IIRN',
 'IMAGE',
 'IU',
 'KYWDS',
 'NAME',
 'NU',
 'ORG',
 'PAGE',
 'PDS',
 'PHYS',
 'PWC',
 'REC',
 'REEL',
 'REELC',
 'REFNO',
 'RPTNO',
 'RSCHR',
 'SEC',
 'SERV',
 'SRCNO',
 'SUBJ',
 'TTL',
 'TYPE',
 'VOL'}

In [6]:
# Write the fields to disk
with open(header_file_name, "w") as f:
    # Loop over each header and write each on a new line
    for field in sorted(all_fields):
        f.write(field + "\n")
print("Wrote {}".format(header_file_name))

Wrote headers.txt


## Load Headers

* Now that we have saved the headers to disk we can load this file without having to parse all of the files ever again!
* If there are a lot of files this is a huge time saver
* This cell opens the header file and creates a list of the column headers for the CSV file
* It also does some re-ordering to put identifying information in the leftmost columns

In [ ]:
with open(header_file_name, "r") as f:
    header_fields = [line.strip() for line in f]
    
    # move REC to beginning of list
    REC_index = header_fields.index('REC')
    header_fields.insert(0, header_fields.pop(REC_index))
    
    # add BATCH & ID columns
    header_fields.insert(0, "BATCH")
    header_fields.insert(0, "ID")

# Print the list for good measure
print(header_fields)
print("There are {} columns in the data.".format(len(header_fields)))

## Parse Data Files

* Ok, now we are at the real meat of the program, the code that parses the data files and transforms it into a CSV file
* Because we are opening files and looping it is difficult to split this program into separate Jupyter cells (which is nice for exposition).

### The Logic of this program:

* Open a CSV file for writing
* Write the headers to the file
* Loop over each data file
    * Open the file
    * Parse the first line (which contains a REC field)
    * Create a dictionary for the row
    * Populate the REC, ID, and BATCH fields
    * Loop over every line in the file
        * Parse the line
        * If the line is REC
            * Write the row to the CSV file
            * Create a new row dictionary
        * If the line is extra data
            * Concatinate the new data to existing values
        * Else 
            * Add the data to the row dictionary
    * Write the last row to the CSV file

In [ ]:
# Open a data file for writing
with open(data_file_name, "w") as csvfile:
    
    # Create a DictWriter instead of CSVWriter because it is better for this use case
    writer = csv.DictWriter(csvfile, fieldnames=header_fields)
    
    # Write the headers to the file
    writer.writeheader()
    

    # Loop over all of the files 
    for file in file_list:
        
        # Display which file is being processed
        print("Processing file {}".format(file))
        
        # Open the QuadraSTAR file for reading
        with open(file, "r") as f:
            
            # get the id from the file name
            file_name = file.stem
        
            # read the first line of the file 
            # which should be REC
            first_line = f.readline()
            
            # split the line based on fixed field lengths
            # Strings can be sliced
            header, value = first_line[0:6], first_line[7:]

            # handle the "--REC--" case by replacing dashes with nothing
            # strip extra whitespace and newlines
            header = header.replace("-", " ").strip()

            # remove newlines from the values
            value = value.strip()

            # Create a dictionary 
            row_dictionary = { field:"" for field in header_fields}
            
            # Add columns for REC, ID, and BATCH
            row_dictionary[header] = value
            row_dictionary['ID'] = file_name + "_" + value
            row_dictionary['BATCH'] = file_name
                        
            # loop over lines in file
            for line in f:

                 # split the line based on fix field lengths
                header, value = line[0:6], line[7:]

                # handle the "--REC--" case and strip newlines
                header = header.replace("-", " ").strip()
                
                # remove newlines from the values
                value = value.strip()

                # when we get to a new record
                if header == "REC": 
                    # save the old record to disk
                    writer.writerow(row_dictionary)
                    # create a new empty row dictionary
                    row_dictionary = { field:"" for field in header_fields}
                    # put this REC in the new row dictionary
                    row_dictionary[header] = value
                    # add ID
                    row_dictionary['ID'] = file_name + "_" + value
                    # add Batch
                    row_dictionary['BATCH'] = file_name
              
                # Check to see if an entry already exists in the dictionary
                elif (header in row_dictionary) and row_dictionary[header] and (value != row_dictionary[header]):
                    
                    # append new value, separated by semi-colon
                    row_dictionary[header] = row_dictionary[header] + "; " + value
                
                # This isn't a new or existing record, populate the dictionary
                else: 
                    
                    # populate the row_dictionary with values 
                    row_dictionary[header] = value
            
            # Write the last record to the CSV file
            writer.writerow(row_dictionary)

# Display completed message   
print("Transformation complete.")